In [4]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from math import sqrt
import matplotlib.pyplot as plt

In [10]:
data=pd.read_csv('C:/Users/Yixuan/Desktop/Madison/stat628/module2/BodyFat.csv')
data.set_index(["IDNO"], inplace=True)

In [11]:
data.head()

,BODYFAT,DENSITY,AGE,WEIGHT,HEIGHT,ADIPOSITY,NECK,CHEST,ABDOMEN,HIP,THIGH,KNEE,ANKLE,BICEPS,FOREARM,WRIST
IDNO,,,,,,,,,,,,,,,,
1,12.6,1.0708,23,154.25,67.75,23.7,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
2,6.9,1.0853,22,173.25,72.25,23.4,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
3,24.6,1.0414,22,154.00,66.25,24.7,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
4,10.9,1.0751,26,184.75,72.25,24.9,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
5,27.8,1.0340,24,184.25,71.25,25.6,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


Adiposity=BMI index
Weight and height are use another system to measure with lbs and inches
Others are measured by cm

In [12]:
data.describe()

,BODYFAT,DENSITY,AGE,WEIGHT,HEIGHT,ADIPOSITY,NECK,CHEST,ABDOMEN,HIP,THIGH,KNEE,ANKLE,BICEPS,FOREARM,WRIST
count,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,18.938492,1.055574,44.884921,178.924405,70.148810,25.436905,37.992063,100.824206,92.555952,99.904762,59.405952,38.590476,23.102381,32.273413,28.663889,18.229762
std,7.750856,0.019031,12.602040,29.389160,3.662856,3.648111,2.430913,8.430476,10.783077,7.164058,5.249952,2.411805,1.694893,3.021274,2.020691,0.933585
min,0.000000,0.995000,22.000000,118.500000,29.500000,18.100000,31.100000,79.300000,69.400000,85.000000,47.200000,33.000000,19.100000,24.800000,21.000000,15.800000
25%,12.800000,1.041400,35.750000,159.000000,68.250000,23.100000,36.400000,94.350000,84.575000,95.500000,56.000000,36.975000,22.000000,30.200000,27.300000,17.600000
50%,19.000000,1.054900,43.000000,176.500000,70.000000,25.050000,38.000000,99.650000,90.950000,99.300000,59.000000,38.500000,22.800000,32.050000,28.700000,18.300000
75%,24.600000,1.070400,54.000000,197.000000,72.250000,27.325000,39.425000,105.375000,99.325000,103.525000,62.350000,39.925000,24.000000,34.325000,30.000000,18.800000
max,45.100000,1.108900,81.000000,363.150000,77.750000,48.900000,51.200000,136.200000,148.100000,147.700000,87.300000,49.100000,33.900000,45.000000,34.900000,21.400000


how to deal with the bodyfat 0?

Using BMI to check the obsercation

In [41]:
# 定义从输入数据集中取指定列作为训练集和测试集的函数(从取1列一直到取11列):
def xattrSelect(x, idxSet):
    xOut = []
    for row in x:
        xOut.append([row[i] for i in idxSet])
    return(xOut)

In [42]:
xList = []  # 构造用于存放属性集的列表
labels = [float(label) for label in data.iloc[:,0].tolist()]  # 提取出data中的标签集并放入列表中
names = data.columns.tolist()          # 提取出data中所有属性的名称并放入列表中
for i in range(len(data)):
    xList.append(data.iloc[i,1:])    # 列表xList中的每个元素对应着data中除去标签列的每一行

In [43]:
# 将原始数据集划分成训练集(占2/3)和测试集(占1/3)：
indices = range(len(xList))
xListTest = [xList[i] for i in indices if i%3 == 0 ]
xListTrain = [xList[i] for i in indices if i%3 != 0 ]
labelsTest = [labels[i] for i in indices if i%3 == 0]
labelsTrain = [labels[i] for i in indices if i%3 != 0]

In [50]:
attributeList = []            # 构造用于存放属性索引的列表
index = range(len(xList[1]))  # index用于下面代码中的外层for循环
indexSet = set(index)         # 构造由names中的所有属性对应的索引构成的索引集合
oosError = []                 # 构造用于存放下面代码中的内层for循环每次结束后最小的RMSE

In [51]:
for i in index:
    attSet = set(attributeList)
    attTrySet = indexSet - attSet          # 构造由不在attributeList中的属性索引组成的集合
    attTry = [ii for ii in attTrySet]      # 构造由在attTrySet中的属性索引组成的列表
    errorList = []
    attTemp = []
    
    for iTry in attTry:
        attTemp = [] + attributeList
        attTemp.append(iTry)
        
        # 调用attrSelect函数从xListTrain和xListTest中选取指定的列构成暂时的训练集与测试集
        xTrainTemp = xattrSelect(xListTrain, attTemp)
        xTestTemp = xattrSelect(xListTest, attTemp)
        
        # 将需要用到的训练集和测试集都转化成数组对象
        xTrain = np.array(xTrainTemp) 
        yTrain = np.array(labelsTrain)
        xTest = np.array(xTestTemp)
        yTest = np.array(labelsTest)
        
        # 使用scikit包训练线性回归模型
        Model = linear_model.LinearRegression()
        Model.fit(xTrain,yTrain)
        
        # 计算在测试集上的RMSE
        rmsError = np.linalg.norm((yTest-Model.predict(xTest)), 2)/sqrt(len(yTest)) # 利用向量的2范数计算RMSE
        errorList.append(rmsError)
        attTemp = []
        
    iBest = np.argmin(errorList)          # 选出errorList中的最小值对应的新索引
    attributeList.append(attTry[iBest])   # 利用新索引iBest将attTry中对应的属性索引添加到attributeList中
    oosError.append(errorList[iBest])     # 将errorList中的最小值添加到oosError列表中
 
print("Out of sample error versus attribute set size" )
print(oosError)
print("" + "Best attribute indices")
print(attributeList)
namesList = [names[i+1] for i in attributeList]
print("" + "Best attribute names")
print(namesList)

Out of sample error versus attribute set size
[0.4825139820055133, 0.4828458946884895, 0.49164672866213127, 0.5057082947910271, 0.5119424374581361, 0.5117510034458413, 0.5293261633285891, 0.5440923469918463, 0.5497842393660439, 0.5558648162540739, 0.5847238796838584, 0.5952843929494762, 0.6084084788693533, 0.6268493182919517, 0.6439340954126517]
Best attribute indices
[0, 1, 3, 11, 9, 12, 10, 14, 5, 13, 4, 8, 7, 2, 6]
Best attribute names
['DENSITY', 'AGE', 'HEIGHT', 'ANKLE', 'THIGH', 'BICEPS', 'KNEE', 'WRIST', 'NECK', 'FOREARM', 'ADIPOSITY', 'HIP', 'ABDOMEN', 'WEIGHT', 'CHEST']
